In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("twitter_transformation").getOrCreate()

23/09/22 16:14:27 WARN Utils: Your hostname, bebeto-HP-EliteDesk-800-G4-DM-35W-Brazil resolves to a loopback address: 127.0.1.1; using 192.168.18.83 instead (on interface wlp2s0)
23/09/22 16:14:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 16:14:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.json("../../datalake/twitter_datascience")

In [4]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{47, 4, 2023-09-...|{[{2023-09-19T02:...|{1234567890abcdef}|  2023-09-19|
|[{78, 48, 2023-09...|{[{2023-09-19T06:...|              null|  2023-09-19|
|[{94, 10, 2023-09...|{[{2023-09-21T09:...|              null|  2023-09-22|
|[{9, 58, 2023-09-...|{[{2023-09-20T02:...|              null|  2023-09-20|
|[{99, 56, 2023-09...|{[{2023-09-21T05:...|              null|  2023-09-21|
+--------------------+--------------------+------------------+------------+



In [5]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [6]:
from pyspark.sql import functions as f

In [7]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [8]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{47, 4, 2023-09-1...|
|{29, 62, 2023-09-...|
|{91, 69, 2023-09-...|
|{31, 57, 2023-09-...|
|{73, 66, 2023-09-...|
|{64, 18, 2023-09-...|
|{9, 45, 2023-09-1...|
|{13, 61, 2023-09-...|
|{44, 44, 2023-09-...|
|{4, 87, 2023-09-1...|
|{78, 48, 2023-09-...|
|{77, 79, 2023-09-...|
|{6, 21, 2023-09-1...|
|{1, 52, 2023-09-1...|
|{29, 71, 2023-09-...|
|{56, 100, 2023-09...|
|{52, 95, 2023-09-...|
|{14, 74, 2023-09-...|
|{44, 87, 2023-09-...|
|{61, 24, 2023-09-...|
+--------------------+
only showing top 20 rows



In [9]:
tweet_df = df.select(f.explode("data").alias("tweets")).select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [10]:
tweet_df.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [11]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       47|              4|2023-09-19T00:10:...| 78|        85|         82|         25|           39|Tweet fictício ge...|
|       29|             62|2023-09-19T18:44:...| 57|        73|         79|         92|           99|Outro tweet fictí...|
|       91|             69|2023-09-19T13:39:...| 31|        61|         80|         83|           81|Tweet fictício ge...|
|       31|             57|2023-09-19T11:39:...| 14|        17|         44|          7|            6|Tweet fictício ge...|
|       73|             66|2023-09-19T01:01:...| 64|        98|         13|         79|           41|Tweet fictício cr...|
+---------+-----

In [12]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [13]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [14]:
users_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [15]:
users_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2023-09-19T02:09:...| 20|User 1|   user1|
|2023-09-19T02:48:...| 13|User 2|   user2|
|2023-09-19T06:32:...| 86|User 3|   user3|
|2023-09-19T21:20:...| 76|User 4|   user4|
|2023-09-19T08:42:...| 28|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [16]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
users_df.coalesce(1).write.mode("overwrite").json('output/user')